In [6]:
import os
from Crypto.Util.Padding import pad, unpad
from datetime import datetime, timedelta
from Crypto.Cipher import AES
from os import urandom
import requests
from Crypto.Util.number import long_to_bytes, bytes_to_long
import json
from tqdm import tqdm
import re
def json_get(url):
    return json.loads(requests.get(url).content.decode())
# safe
def bytes_to_words(b):
    return [int.from_bytes(b[i:i+4], 'little') for i in range(0, len(b), 4)]

# safe
def rotate(x, n):
    return ((x << n) & 0xffffffff) | ((x >> (32 - n)) & 0xffffffff)

# safe
def word(x):
    return x % (2 ** 32)

# safe
def words_to_bytes(w):
    return b''.join([i.to_bytes(4, 'little') for i in w])

# safe
def xor(a, b):
    return b''.join([bytes([x ^ y]) for x, y in zip(a, b)])
def encrypt():
    return bytes.fromhex(json_get(f"https://aes.cryptohack.org/stream_consciousness/encrypt/")['ciphertext'])
def is_utf8(s: str):
    try:
        s.decode('utf-8')
        return True
    except UnicodeError:
        return False
KEY =  b"key{the_aes_key}"
FLAG = b"crypto{aes_flag}"
PLAIN =b"Howdy! I'm Flowe"
key = urandom(32)
iv = urandom(16)

In [10]:
from chacha20_c76e2a43164af8b6ecb0e145415ab931 import ChaCha20
c = ChaCha20()



In [12]:
c.decrypt(c.encrypt(msg_plain[:64], key, iv), key, iv)

[1634760805, 857760878, 2036477234, 1797285236, 2824785405, 3029413874, 36017240, 2175624009, 3123643198, 2037031742, 3601762572, 2687753134, 1, 3540185883, 609254888, 4063375929, 3150916580]
[1348577397, 2038877622, 3553777064, 3164987645, 4156322200, 1982720138, 762850976, 2352592029, 2828247835, 2933175598, 1169941739, 2184147724, 1922636911, 3103196146, 3978973237, 2182822408, 3150916580]
[1634760805, 857760878, 2036477234, 1797285236, 2824785405, 3029413874, 36017240, 2175624009, 3123643198, 2037031742, 3601762572, 2687753134, 1, 3540185883, 609254888, 4063375929, 3150916580]
[1348577397, 2038877622, 3553777064, 3164987645, 4156322200, 1982720138, 762850976, 2352592029, 2828247835, 2933175598, 1169941739, 2184147724, 1922636911, 3103196146, 3978973237, 2182822408, 3150916580]


b'Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean'

In [ ]:
print(bytes_to_words(xor(msg_enc[:64], msg_plain[:64])))

[3217604799, 516298735, 650097383, 1317965626, 3858257107, 930165007, 3884825833, 1863269409, 3272252913, 1106575246, 2249602672, 3547854353, 659308596, 1226940503, 3733916889, 1330498663]


In [5]:
print(bytes_to_words(xor(msg_enc[64:], msg_plain[64:])))

[1271567934, 1714120261, 4121912195, 1359301054]


In [11]:
xor(b"crypto{", flag_enc)

b'\xb62~\x9a"S\x03@\x964J\xd5iJ @\xb1\x14u>$\xea\x9c\x1a\xf1|\x10&2\x81\x98}\x9fR\xc7HI'

In [3]:
# GPT-4o mini
def inv_rotate(x, n):
    return ((x >> n) & 0xffffffff) | ((x << (32 - n)) & 0xffffffff)
def rotate(x, n):
    return ((x << n) & 0xffffffff) | ((x >> (32 - n)) & 0xffffffff)
def word(a ,b):
    return ((a+b) % (2 ** 32), b)
def inv_word(a, b):
    return ((a - b) % (2 ** 32), b)
def quarter_round(x, a, b, c, d):
    x[a], x[b] = word(x[a], x[b]); x[d] ^= x[a]; x[d] = rotate(x[d], 16)
    x[c], x[d] = word(x[c], x[d]); x[b] ^= x[c]; x[b] = rotate(x[b], 12)
    x[a], x[b] = word(x[a], x[b]); x[d] ^= x[a]; x[d] = rotate(x[d], 8)
    x[c], x[d] = word(x[c], x[d]); x[b] ^= x[c]; x[b] = rotate(x[b], 7)
def inv_quarter_round(x, a, b, c, d):
    # Step 4: Reverse the last block (rotate by 7)
    x[b] = inv_rotate(x[b], 7)
    x[b] ^= x[c]
    x[c], x[d] = inv_word(x[c], x[d])

    # Step 3: Reverse the third block (rotate by 8)
    x[d] = inv_rotate(x[d], 8)
    x[d] ^= x[a]
    x[a], x[b] = inv_word(x[a], x[b])

    # Step 2: Reverse the second block (rotate by 12)
    x[b] = inv_rotate(x[b], 12)
    x[b] ^= x[c]
    x[c], x[d] = inv_word(x[c], x[d])

    # Step 1: Reverse the first block (rotate by 16)
    x[d] = inv_rotate(x[d], 16)
    x[d] ^= x[a]
    x[a], x[b] = inv_word(x[a], x[b])

def inner_block(state):
    quarter_round(state, 0, 4, 8, 12)
    quarter_round(state, 1, 5, 9, 13)
    quarter_round(state, 2, 6, 10, 14)
    quarter_round(state, 3, 7, 11, 15)
    quarter_round(state, 0, 5, 10, 15)
    quarter_round(state, 1, 6, 11, 12)
    quarter_round(state, 2, 7, 8, 13)
    quarter_round(state, 3, 4, 9, 14)

def inv_inner_block(state):
    # Reverse the quarter rounds in the exact reverse order

    # Last block in original _inner_block
    inv_quarter_round(state, 3, 4, 9, 14)
    inv_quarter_round(state, 2, 7, 8, 13)
    inv_quarter_round(state, 1, 6, 11, 12)
    inv_quarter_round(state, 0, 5, 10, 15)

    # Second-to-last block in original _inner_block
    inv_quarter_round(state, 3, 7, 11, 15)
    inv_quarter_round(state, 2, 6, 10, 14)
    inv_quarter_round(state, 1, 5, 9, 13)
    inv_quarter_round(state, 0, 4, 8, 12)
# safe
def words_to_bytes(w):
    return b''.join([i.to_bytes(4, 'little') for i in w])
# safe
def xor(a, b):
    return b''.join([bytes([x ^ y]) for x, y in zip(a, b)])
# safe
def bytes_to_words(b):
    return [int.from_bytes(b[i:i+4], 'little') for i in range(0, len(b), 4)]

In [18]:
msg_enc = bytes.fromhex("f3afbada8237af6e94c7d2065ee0e221a1748b8c7b11105a8cc8a1c74253611c94fe7ea6fa8a9133505772ef619f04b05d2e2b0732cc483df72ccebb09a92c211ef5a52628094f09a30fc692cb25647f")
msg_plain = b'Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula.'
flag_enc = bytes.fromhex("b6327e9a2253034096344ad5694a2040b114753e24ea9c1af17c10263281fb0fe622b32732")
state_msgb1 = bytes_to_words(xor(msg_enc[:64], msg_plain[:64]))
iv1 = bytes.fromhex('e42758d6d218013ea63e3c49')
iv2 = bytes.fromhex('a99f9a7d097daabd2aa2a235')
const = [0x61707865, 0x3320646e, 0x79622d32, 0x6b206574]
counter = 1
for j in range(10):
    inv_inner_block(state_msgb1)
rev_const, rev_key, rev_counter, rev_iv = state_msgb1[:4], words_to_bytes(state_msgb1[4:12]), state_msgb1[12], words_to_bytes(state_msgb1[13:])
assert rev_const == const
assert rev_counter == counter
assert iv1 == rev_iv
from chacha20_c76e2a43164af8b6ecb0e145415ab931 import ChaCha20
c = ChaCha20()
c.decrypt(flag_enc, rev_key, iv2)

[1634760805, 857760878, 2036477234, 1797285236, 269811001, 1531508478, 2693662451, 4136581883, 2202295096, 473046972, 3423948778, 1816740636, 1, 2107285417, 3182066953, 899850794]
[3926343893, 225983574, 3145419943, 1884427615, 185694916, 703520379, 1350454702, 1053618945, 109072004, 1722994255, 905922039, 2369734823, 1020269484, 2307568854, 2452136909, 3401202256]


b'crypto{M1x1n6_r0und5_4r3_1nv3r71bl3!}'